<a href="https://colab.research.google.com/github/asmaakhaledd/PID-NN/blob/Dataset-trial/PID-NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install numpy pandas tensorflow control matplotlib xmltodict scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.6/549.6 kB 7.4 MB/s eta 0:00:00


In [5]:
from google.colab import drive
drive.mount('/content/drive')

data_path = "/content/drive/My Drive/OhioT1/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import xmltodict
import pandas as pd
import os

def parse_xml(file_path):
    with open(file_path, 'r') as file:
        data_dict = xmltodict.parse(file.read())
    return data_dict

# Extract relevant fields (Glucose, Insulin, Meals, etc.)
def extract_features(data_dict):
    patient = data_dict['patient']

    if 'glucose_level' in patient:
        for event in patient['glucose_level']['event']:
            entry = {
                'time': event['@ts'],
                'glucose': float(event['@value']),
                'insulin': 0,
                'meal': 0
            }
            patient_data.append(entry)

     # Insulin events
    if 'bolus' in patient:
        for event in patient['bolus']['event']:
            entry = {
                'time': event['@ts'],
                'glucose': 0,
                'insulin': float(event['@value']),
                'meal': 0
            }
            patient_data.append(entry)

     # Insulin events
    if 'basal' in patient:
        for event in patient['basal']['event']:
            entry = {
                'time': event['@ts'],
                'glucose': 0,
                'insulin': float(event['@value']),
                'meal': 0
            }
            patient_data.append(entry)

       # Insulin events
    if 'bolus' in patient:
        for event in patient['bolus']['event']:
            entry = {
                'time': event['@ts'],
                'glucose': 0,
                'insulin': float(event['@value']),
                'meal': 0
            }
            patient_data.append(entry)

    temp_basal

       # Insulin events (if present)
    if 'temp_basal' in patient:
        for event in patient['temp_basal']['event']:
            entry = {
                'time': event['@ts'],
                'glucose': 0,
                'insulin': float(event['@value']),
                'meal': 0
            }
            patient_data.append(entry)

    return pd.DataFrame(patient_data)

# Process all XML files in a folder
all_dataframes = []
for file in os.listdir(data_path):
    if file.endswith(".xml"):
        data_dict = parse_xml(os.path.join(data_path, file))
        df = extract_features(data_dict)
        all_dataframes.append(df)

# Combine all patients' data
df = pd.concat(all_dataframes, ignore_index=True)

# Convert time column to datetime
df['time'] = pd.to_datetime(df['time'])
df.fillna(0, inplace=True)  # Replace NaN values with 0
df.sort_values(by='time', inplace=True)

print(df.head())  # Preview dataset


KeyError: 'patient_data'

Convert XML to Pandas DataFrame

In [ ]:
import xmltodict
import pandas as pd
import os

def parse_xml(file_path):
    with open(file_path, 'r') as file:
        data_dict = xmltodict.parse(file.read())
    return data_dict

# Extract only glucose, insulin, and meals (ignoring exercise, sleep, and stress)
def extract_features(data_dict):
    patient_data = []

    for event in data_dict['patient_data']['events']['event']:
        entry = {
            'time': event['@ts'],
            'glucose': event.get('glucose_level', None),
            'insulin': event.get('bolus', None),
            'meal': event.get('meal', None),
        }
        patient_data.append(entry)

    return pd.DataFrame(patient_data)

# Process all XML files in a folder
all_dataframes = []
for file in os.listdir(data_path):
    if file.endswith(".xml"):
        data_dict = parse_xml(os.path.join(data_path, file))
        df = extract_features(data_dict)
        all_dataframes.append(df)

# Combine all patients' data
df = pd.concat(all_dataframes, ignore_index=True)

# Convert time column to datetime
df['time'] = pd.to_datetime(df['time'])
df.fillna(0, inplace=True)  # Replace NaN values with 0
df.sort_values(by='time', inplace=True)

print(df.head())  # Preview dataset

Prepare Data for LSTM Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Use only glucose, insulin, and meal as input features
features = ['glucose', 'insulin', 'meal']
target = ['Kp', 'Ki', 'Kd']  # PID gains

# Normalize the features
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

# Generate PID controller gains dynamically (example logic)
df['Kp'] = df['glucose'] * 0.01
df['Ki'] = df['glucose'] * 0.001
df['Kd'] = df['glucose'] * 0.0001

# Create sequences for LSTM (time-series format)
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length][features].values)
        y.append(data.iloc[i+seq_length][target].values)
    return np.array(X), np.array(y)

seq_length = 10  # Use past 10 timestamps for prediction
X, y = create_sequences(df, seq_length)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Train the LSTM Neural Network

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define LSTM Model
model = Sequential([
    LSTM(64, activation='tanh', return_sequences=True, input_shape=(seq_length, len(features))),
    Dropout(0.2),
    LSTM(32, activation='tanh', return_sequences=False),
    Dense(16, activation='relu'),
    Dense(3, activation='linear')  # Outputs: Kp, Ki, Kd
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Implement PID Controller

In [ ]:
import control as ctrl

# Predict PID parameters using trained LSTM model
sample_input = X_test[0].reshape(1, seq_length, len(features))
Kp, Ki, Kd = model.predict(sample_input)[0]

# Define PID Controller
pid = ctrl.TransferFunction([Kd, Kp, Ki], [1, 0])  # PID equation

# Define a system response model
system = ctrl.tf([1], [1, 2, 1])  # Example glucose-insulin system

# Closed-loop system
closed_loop = ctrl.feedback(pid * system)

# Simulate step response
import matplotlib.pyplot as plt

time, response = ctrl.step_response(closed_loop)
plt.plot(time, response)
plt.xlabel('Time')
plt.ylabel('Glucose Level')
plt.title('LSTM-Based Neural Network PID Glucose Control')
plt.show()


Real-Time PID Tuning

In [ ]:
import time

while True:
    # Simulate real-time glucose level reading
    real_glucose = np.random.uniform(60, 180)  # Simulated value
    real_input = np.array([[real_glucose, 0, 0]])  # Only glucose, insulin, meal
    real_input = scaler.transform(real_input)
    real_input = real_input.reshape(1, seq_length, len(features))

    # Get PID parameters from LSTM model
    Kp, Ki, Kd = model.predict(real_input)[0]

    print(f"Real-Time Glucose: {real_glucose:.2f}, Kp: {Kp:.4f}, Ki: {Ki:.4f}, Kd: {Kd:.4f}")

    time.sleep(5)  # Simulate real-time updates every 5 seconds
